This notebook tests whether the results of the scoring and extraction of field magnitude values from parcels makes sense with what we know about the general properties of the field simulation.

In this test we perform label separation of the parcel ribbon, select the same target area in this new parcellation scheme and ensure locality properties of the resulting score values. 

In [5]:
import os
import gmsh
import nibabel as nib
import numpy as np
from nilearn import plotting, image, regions
from fieldopt import tetrapro, geolib

## Step 1: Pulling Spatially Disparate Parcels

In [2]:
#Get parcels
ribbon_dir = '../../data/sub-CMH090/ribbon/'
t1 = '../../data/simnibs_output/m2m_sub-CMH090/T1fs_nu_conform.nii.gz'
L_ribbons = sorted([os.path.join(ribbon_dir,f) for f in os.listdir(ribbon_dir) if '.L.network' in f])
R_ribbons = sorted([os.path.join(ribbon_dir,f) for f in os.listdir(ribbon_dir) if '.R.network' in f])

In [3]:
L_img = image.load_img(L_ribbons[0])
t1_img = image.load_img(t1)

In [ ]:
#View left ribbon
plotting.view_img(L_img,bg_img=t1_img)

In [4]:
#Pull DLPFC coordinates (label = 4)
L_data = L_img.get_data()
L_dlpfc_inds = np.where(L_data == 4)
L_dlpfc_data = np.zeros_like(L_data)
L_dlpfc_data[L_dlpfc_inds] = 1
L_dlpfc_img = nib.Nifti1Image(L_dlpfc_data,affine=L_img.affine)

In [ ]:
#View
plotting.view_img(L_dlpfc_img,bg_img=t1_img)

In [6]:
#Perform the same operation on R side (DLPFC = 11)
R_img = image.load_img(R_ribbons[0])
R_data = R_img.get_data()
R_dlpfc_inds = np.where(R_data == 11)
R_dlpfc_data = np.zeros_like(R_data)
R_dlpfc_data[R_dlpfc_inds] = 1
R_dlpfc_img = nib.Nifti1Image(R_dlpfc_data,affine=R_img.affine)

In [ ]:
plotting.view_img(R_dlpfc_img, bg_img=t1_img)

In [7]:
#Combine DLPFC images into 1
dlpfc_img = image.math_img('a + (2*b)', a=L_dlpfc_img,b=R_dlpfc_img)

## Step 2: Perform Tetrahedral Projection

In [8]:
msh_file = '../../data/simnibs_output/sub-CMH090.msh'

In [9]:
#Constants
TET = (3,2)
GM = (2,2)
WM = (2,1)

In [10]:
#Load in relevant data
_, _, tet_param = geolib.load_gmsh_elems(msh_file, TET)
tet_node, tet_coord, _ = geolib.load_gmsh_nodes(msh_file, TET)
gm_node, gm_coord, _ = geolib.load_gmsh_nodes(msh_file, GM)
wm_node, wm_coord, _ = geolib.load_gmsh_nodes(msh_file, WM)

In [11]:
#Formulate lists as arrays
n_nodes = len(tet_param[0])//4
tet_node_list = np.array(tet_param[0]).reshape( (n_nodes,4) )

In [12]:
#Generate array storing node properties
prop_arr = np.array([
    [min(tet_node), max(tet_node), len(tet_node)],
    [min(gm_node), max(gm_node), len(gm_node)],
    [min(wm_node), max(wm_node), len(wm_node)]
])

In [26]:
#Make inputs to tetrahedral projection
node_list = tetrapro.map_nodes(tet_node_list, prop_arr)
coord_arr = np.concatenate((tet_coord.flatten(),
                            gm_coord.flatten(),
                            wm_coord.flatten()))
ribbon = dlpfc_img.get_data()
affine = dlpfc_img.affine

In [ ]:
#Run projection
project_arr = tetrapro.tetrahedral_projection(node_list,
                                             coord_arr,
                                             ribbon,
                                             affine)

## Step 3: Compute scores on each hemisphere